# Imports

In [1]:
import sys
sys.path.append(r'D:\Programming\3rd_party\keras')

In [2]:
import sys
from imp import reload
import numpy as np
import keras

from keras.models import Model, load_model
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

import tensorflow as tf

c:\miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data

In [3]:
import load_data
load_data = reload(load_data)

In [4]:
DEV_MODE_RANGE = 0 # off

In [5]:
train_df = load_data.LoadData(train_data = True, DEV_MODE_RANGE = DEV_MODE_RANGE)

(3920, 9) (18000, 1) (22000, 1)


HBox(children=(IntProgress(value=0, max=3920), HTML(value='')))

HBox(children=(IntProgress(value=0, max=3920), HTML(value='')))

In [6]:
test_fold_no = 1

In [7]:
train_images, train_masks, validate_images, validate_masks = load_data.SplitTrainData(train_df, test_fold_no)
train_images.shape, train_masks.shape, validate_images.shape, validate_masks.shape

((3136,), (3136,), (784,), (784,))

# Reproducability setup:

In [8]:
import random as rn

kSeed = 241075

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(kSeed)
rn.seed(kSeed)

#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(kSeed)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
sess = tf.Session(graph=tf.get_default_graph())
K.set_session(sess)

# IOU metric

In [9]:
thresholds = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])

def iou(img_true, img_pred):
    i = np.sum((img_true*img_pred) >0)
    u = np.sum((img_true + img_pred) >0)
    if u == 0:
        return u
    return i/u

def iou_metric(imgs_true, imgs_pred):
    num_images = len(imgs_true)
    scores = np.zeros(num_images)
    
    for i in range(num_images):
        if imgs_true[i].sum() == imgs_pred[i].sum() == 0:
            scores[i] = 1
        else:
            scores[i] = (thresholds <= iou(imgs_true[i], imgs_pred[i])).mean()
            
    return scores.mean()

def iou_metric_batch(y_true_in, y_pred_in):
    y_pred_in = y_pred_in > 0.5 # added by sgx 20180728
    batch_size = y_true_in.shape[0]
    metric = []
    for batch in range(batch_size):
        value = iou_metric(y_true_in[batch], y_pred_in[batch])
        metric.append(value)
    #print("metric = ",metric)
    return np.mean(metric)

# adapter for Keras
def my_iou_metric(label, pred):
    metric_value = tf.py_func(iou_metric_batch, [label, pred], tf.float64)
    return metric_value


# Data generator

In [10]:
mean_val = np.mean(train_images.apply(np.mean))
mean_std = np.mean(train_images.apply(np.std))
mean_val, mean_std 

(0.4823356019939591, 0.11072641306201825)

In [11]:
sys.path.insert(1, '../3rd_party/albumentations')
sys.path.insert(1, '../3rd_party/imgaug')
import albumentations

In [12]:
augmented_image_size = 303
nn_image_size = 224

In [13]:
def basic_aug(p=1.):
    return albumentations.Compose([
        albumentations.Resize(augmented_image_size, augmented_image_size),
        albumentations.HorizontalFlip(),
        albumentations.RandomCrop(nn_image_size, nn_image_size),
        albumentations.Normalize(mean = mean_val, std = mean_std, max_pixel_value = 1.0),
    ], p=p)

In [14]:
import threading
class AlbuDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, images, masks, batch_size=32, nn_image_size = 96, shuffle=True):
        'Initialization'
        self.images = images
        self.masks = masks
        self.batch_size = batch_size
        self.nn_image_size = nn_image_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.images))
        self.on_epoch_end()
        self.augmentation = basic_aug()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.nn_image_size,self.nn_image_size, 3), dtype=np.float32)
        y = np.empty((self.batch_size, self.nn_image_size,self.nn_image_size, 1), dtype=np.float32)

        # Generate data
        for i, index in enumerate(indexes):
            
            image = self.images[index]
            mask = None if self.masks is None else self.masks[index]
            aug_res = self.augmentation(image = image, mask = mask)
            image = aug_res['image']
            X[i, ...] = image
            mask = aug_res['mask']
            y[i, ...] = mask.reshape(mask.shape[0], mask.shape[1], 1)

        return X, y

# model

In [15]:
sys.path.append('../3rd_party/segmentation_models')
import segmentation_models
segmentation_models = reload(segmentation_models)
from segmentation_models.utils import set_trainable


In [16]:
model_name = 'FNN'
backbone_name='resnet152'

In [17]:
model = None
if model_name == 'FNN':
    model = segmentation_models.FPN(backbone_name=backbone_name, input_shape=(nn_image_size, nn_image_size, 3), encoder_weights='imagenet', freeze_encoder=True)


In [18]:
model_out_file = 'models_1/{model_name}_{backbone_name}_{test_fold_no}.model'.format(model_name=model_name, backbone_name=backbone_name, test_fold_no=test_fold_no)
model_out_file

'models_1/FNN_resnet152_1.model'

# Train

In [19]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc", my_iou_metric]) #, my_iou_metric

In [20]:
epochs_warmup = 2
epochs = 250
batch_size = 20

In [21]:
train_gen = AlbuDataGenerator(train_images, train_masks, batch_size=batch_size, nn_image_size = nn_image_size, shuffle=True)
val_gen = AlbuDataGenerator(validate_images, validate_masks, batch_size=batch_size*2, nn_image_size = nn_image_size, shuffle=False)

In [22]:
sys.path.append('../3rd_party/keras-tqdm')
from keras_tqdm import TQDMCallback, TQDMNotebookCallback

In [ ]:
early_stopping = EarlyStopping(monitor='val_my_iou_metric', mode = 'max',patience=30, verbose=1)
model_checkpoint = ModelCheckpoint(model_out_file, monitor='val_my_iou_metric',
                                   mode = 'max', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_my_iou_metric', mode = 'max',factor=0.2, patience=10, min_lr=0.00001, verbose=1)

'''
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath + '.hdf5', save_best_only=True)
    csv_logger = CSVLogger(filepath+'_log.csv', separator=',', append=False)
    return [es, msave, csv_logger]
'''
    


history = model.fit_generator(train_gen,
                    validation_data=val_gen, 
                    epochs=epochs_warmup,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr, TQDMNotebookCallback(leave_inner=True)], # 
                    workers=5,
                    use_multiprocessing=False,
                    verbose=0)

set_trainable(model)

history = model.fit_generator(train_gen,
                    validation_data=val_gen, 
                    epochs=epochs,
                    initial_epoch = epochs_warmup,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr, TQDMNotebookCallback(leave_inner=True)], # 
                    workers=5,
                    use_multiprocessing=False,
                    verbose=0)

HBox(children=(IntProgress(value=0, description='Training', max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=156), HTML(value='')))


Epoch 00001: val_my_iou_metric improved from -inf to 0.42850, saving model to models_1/FNN_resnet152_1.model


HBox(children=(IntProgress(value=0, description='Epoch 1', max=156), HTML(value='')))


Epoch 00002: val_my_iou_metric improved from 0.42850 to 0.59829, saving model to models_1/FNN_resnet152_1.model



...

HBox(children=(IntProgress(value=0, description='Epoch 77', max=156), HTML(value='')))


Epoch 00078: val_my_iou_metric did not improve


HBox(children=(IntProgress(value=0, description='Epoch 78', max=156), HTML(value='')))


Epoch 00079: val_my_iou_metric did not improve


HBox(children=(IntProgress(value=0, description='Epoch 79', max=156), HTML(value='')))